# The Minimax Algorithm

This notebook implements the [minimax algorithm](https://en.wikipedia.org/wiki/Minimax) and thereby implements a program that can play various deterministic, zero-sum, two-person games with perfect information. from Stroetmann

Since Jupyter Nootbook with the NodeJS implemintation has problems loading .ipynb files we will use a plain .js file. All functions of the game are stored ``game`` variable for further use.

Also Jupyter Nootbook has the same issue as in the ``Tic-Tac-Toe Bitboard.ipynb`` so no input can be read. This means that this Notebook is just for documentation and when playing the game ``node minimax.js`` needs to be excuted

In [1]:
const game = require("./Tic-Tac-Toe Bitboard.js");

``numValueCalls``is a util value that stores how often the ``value`` function is called. ``chache`` stores the results of the value function so they can be reused

In [2]:
let numValueCalls = 0;
let cache = {};

The ``other`` function always returns the opposite player of the ``player`` provided 

In [3]:
function other(player) {
    for (let i = 0; i < game.players.length; i++) {
        if (game.players[i] !== player) {
            return game.players[i];
        }
    }
};

``cacheValue`` is a wrapper around the ``value`` function so the results of the calculation can be stored in the ``cache`` variable and reused if the function is called again with the same ``state`` and ``player``

In [4]:
function cacheValue(state, player) {
    if (cache[[state, player]] !== undefined) {
        return cache[[state, player]];
    }
    let val = value(state, player);
    cache[[state, player]] = val;
    return val;
};

The ``value`` function implements the main idea behind the Minimax algorithm and recursivly calls itself to calculate the value for each reachable ``states`` from the curent ``state`` with the active ``player``

In [5]:
function value(state, player) {
    numValueCalls += 1;
    if (game.finished(state)) {
        return game.utility(state, player);
    }
    let o = other(player);
    let result = -1;
    game.nextStates(state, player).forEach(ns => {
        let nextValue = (-cacheValue(ns, o));
        if (nextValue > result) {
            result = nextValue;
        }
    });
    return result;
};

``bestMove`` selects the best possible move for a ``state`` and a ``player``. If there are multiple best moves one of them is randomly choosen. This works by comparing the values of the ``nextStates`` for the current ``state`` and ``player`` with the ``value`` of the current state. Every state that has the same value is added to a list from which wil be choosen.

In [6]:
function bestMove(state, player) {
    let ns = game.nextStates(state, player);
    let bestValue = cacheValue(state, player);
    let bestMoves = [];
    ns.forEach(nextState => {
        if ((-cacheValue(nextState, other(player))) === bestValue) {
            bestMoves.push(nextState);
        }
    });
    let rand = Math.floor(Math.random() * bestMoves.length);
    let bestState = bestMoves[rand];
    return [bestValue, bestState];
};

``playGame`` puts everthing together and uses the ``bestMove`` function to select the best move and then performs it. Afterwards it lets the player input their move and it repeats until either a draw, win or lose is achieved.

In [7]:
function playGame() {
    let state = game.start;
    while (true) {
        let firstPlayer = game.players[0];
        let bm = bestMove(state, firstPlayer);
        let val = bm[0];
        state = bm[1];
        console.log(game.toBoard(state));
        console.log("For me, the game has the value " + val);
        if (game.finished(state)) {
            game.finalMsg(state);
            return;
        }
        state = game.getMove(state);
        console.log(game.toBoard(state));
        if (game.finished(state)) {
            game.finalMsg(state);
            return;
        }
    }
};

This call to ``cacheValue`` calculates the ``value`` for all the states reachable. And is used for the benchmark

In [8]:
let val = cacheValue(game.start, 0);

If both values equal 5478 for TicTacToe the ``cache`` is working perfectly. Which is all the possible states and all entries in the ``cache``

In [9]:
console.log(numValueCalls);
console.log(Object.keys(cache).length);

5478
5478


Call to ``playGame`` to start it. 

In [10]:
//playGame();